In [4]:
import findspark
findspark.init('spark-3.2.0-bin-hadoop3.2')

import pyspark
import random

In [5]:
sc = pyspark.SparkContext(appName='Tarefa 2')

In [6]:
# carregando s datasets para RDDs
flights_file = '/content/datasets/flights.csv'
shakespeare_file = '/content/datasets/shakespeare.txt'
flights_RDD = sc.textFile(flights_file)
shakespeare_RDD = sc.textFile(shakespeare_file)


In [7]:
#Exercício 1a - O tempo total de voo de cada compania

def read_line_time(line):
    record = line.split(',')
    return (record[1],float(record[9]))

time_RDD = flights_RDD.map(read_line_time).reduceByKey(lambda a,b: a+b)
print(time_RDD.collect())

[('19930', 2001295.0), ('20409', 3009096.0), ('20366', 4142864.0), ('20437', 799733.0), ('19690', 449242.0), ('20355', 4194811.0), ('21171', 916098.0), ('19805', 6273366.0), ('19790', 7747256.0), ('20436', 689589.0), ('20398', 2391168.0), ('20304', 3625141.0), ('19977', 6757883.0), ('19393', 9959718.0)]


In [8]:
#Exercício 1b - Qual o destino mais visado

def read_line_destiny(line):
    record = line.split(',')
    return (record[4],1)

destiny_RDD = flights_RDD.map(read_line_destiny).countByKey()
max_key = max(destiny_RDD, key=destiny_RDD.get)
print(str(max_key)+': '+str(destiny_RDD[max_key]))

ATL: 30953


In [16]:
#Exercício 1c - Quais aeroportos cada compania passou

def read_line_airports_origin(line):
  record = line.split(',')
  return (record[1],record[3])

def read_line_airports_destiny(line):
  record = line.split(',')
  return (record[1], record[4])

airports_RDD_origin = flights_RDD.map(read_line_airports_origin)
airports_RDD_destiny = flights_RDD.map(read_line_airports_destiny)

airports_RDD = airports_RDD_origin.union(airports_RDD_destiny).groupByKey()

for comp, airports in airports_RDD.take(5):
  print(comp, sorted(set(airports)))

19930 ['ADK', 'ADQ', 'ANC', 'ATL', 'AUS', 'BET', 'BLI', 'BOS', 'BRW', 'BUR', 'CDV', 'DCA', 'DEN', 'DFW', 'EWR', 'FAI', 'FLL', 'GEG', 'HNL', 'IAH', 'JNU', 'KOA', 'KTN', 'LAS', 'LAX', 'LIH', 'MCI', 'MCO', 'MSP', 'OAK', 'OGG', 'OME', 'ONT', 'ORD', 'OTZ', 'PDX', 'PHL', 'PHX', 'PSG', 'PSP', 'SAN', 'SAT', 'SCC', 'SEA', 'SFO', 'SIT', 'SJC', 'SLC', 'SMF', 'SNA', 'STL', 'TUS', 'WRG', 'YAK']
20409 ['ABQ', 'AUS', 'BDL', 'BOS', 'BQN', 'BTV', 'BUF', 'BUR', 'BWI', 'CHS', 'CLT', 'DCA', 'DEN', 'DFW', 'DTW', 'EWR', 'FLL', 'HOU', 'HPN', 'IAD', 'JAX', 'JFK', 'LAS', 'LAX', 'LGA', 'LGB', 'MCO', 'MSY', 'OAK', 'ORD', 'ORH', 'PBI', 'PDX', 'PHL', 'PHX', 'PIT', 'PSE', 'PVD', 'PWM', 'RDU', 'RIC', 'ROC', 'RSW', 'SAN', 'SAV', 'SEA', 'SFO', 'SJC', 'SJU', 'SLC', 'SMF', 'SRQ', 'STT', 'STX', 'SWF', 'SYR', 'TPA']
21171 ['AUS', 'BOS', 'DCA', 'DFW', 'EWR', 'FLL', 'IAD', 'JFK', 'LAS', 'LAX', 'MCO', 'ORD', 'PDX', 'PHL', 'PSP', 'SAN', 'SEA', 'SFO', 'SJC']
19805 ['ABQ', 'ATL', 'AUS', 'BDL', 'BHM', 'BNA', 'BOS', 'BWI', 'CLE',

In [12]:
#Exercício 1d - O vôo de maior distância de cada companhia

def read_line_airports_distance(line):
  record = line.split(',')
  return record[1], float(record[10])

distance_RDD = flights_RDD.map(read_line_airports_distance).groupByKey()
for comp, distances in distance_RDD.collect():
  print(comp, max(list(distances)))

19930 2874.0
20409 2704.0
20366 1389.0
20437 2139.0
19690 4983.0
20355 2979.0
21171 2704.0
19805 3784.0
19790 4502.0
20436 1703.0
20398 1379.0
20304 1535.0
19977 4962.0
19393 2335.0


In [15]:
#Exercício 1e - Qual é o vôo mais frequente de cada companhia
def read_line_freq(line):
  record = line.split(',')
  return (record[1], int(record[2]))

freq_RDD = flights_RDD.map(read_line_freq).groupByKey()

for comp, freq in freq_RDD.collect():
  freq_list = list(freq)
  print(comp, max(freq_list, key=freq_list.count))

19930 64
20409 2054
20366 2514
20437 221
19690 1
20355 409
21171 251
19805 29
19790 1258
20436 143
20398 2726
20304 5325
19977 1246
19393 583


In [14]:
#Exercício 2 - Utilizando o dataset shakespeare.txt, faça uma operação de MapReduce para construir um 
#índice invertido contendo como chave uma palavra e como valor uma lista de linhas onde 
#ela foi mencionada.

def read_line_text(line):
  line_splited = line.split('\t', 1)
  for word in line_splited[1].strip().replace('\t', ' ').split():
    yield word, line_splited[0]

shakespeare_RDD = sc.textFile(shakespeare_file)
words_RDD = shakespeare_RDD.flatMap(read_line_text).groupByKey()
for word, lines in words_RDD.collect():
  print(word, list(lines))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



A saída de streaming foi truncada nas últimas 5000 linhas.
project ['hamlet@147263', 'antonyandcleopatra@146347', '2kinghenryiv@27316', 'winterstale@102818', 'allswellthatendswell@11218', 'muchadoaboutnothing@53351', 'tempest@42036', 'tempest@82933', 'tempest@99067']
ha't. ['hamlet@147412', 'antonyandcleopatra@61670', 'tamingoftheshrew@123704']
motion ['hamlet@147465', 'coriolanus@56239', 'coriolanus@58937', 'coriolanus@97300', 'cymbeline@31988', 'cymbeline@58929', 'cymbeline@119649', 'troilusandcressida@89849', 'antonyandcleopatra@72117', '2kinghenryiv@86533', 'othello@19369', 'kinghenryv@48196', 'kinghenryviii@18945', 'kinghenryviii@69281', 'winterstale@76561', 'winterstale@141563', '1kinghenryiv@137534', 'romeoandjuliet@61669', 'romeoandjuliet@78978', 'allswellthatendswell@50650', 'merrywivesofwindsor@3175', 'merrywivesofwindsor@63028', 'merrywivesofwindsor@68577', 'measureforemeasure@58371', 'measureforemeasure@70887', 'measureforemeasure@130231', 'loveslabourslost@74817', 'lovesla

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



A saída de streaming foi truncada nas últimas 5000 linhas.
faces] ['timonofathens@64895']
parasites, ['timonofathens@64958', 'venusandadonis@38808']
vapours, ['timonofathens@65107']
Burn, ['timonofathens@65473']
humanity! ['timonofathens@65521']
fury? ['timonofathens@65631']
stones. ['timonofathens@66109']
bankrupts, ['timonofathens@66602']
pill ['timonofathens@66804']
brothel! ['timonofathens@66849']
Piety, ['timonofathens@66949']
neighbourhood, ['timonofathens@67042']
mysteries, ['timonofathens@67088']
Degrees, ['timonofathens@67134']
contraries, ['timonofathens@67176']
Plagues, ['timonofathens@67217']
fevers ['timonofathens@67268']
stroke! ['timonofathens@67308']
sciatica, ['timonofathens@67308']
Cripple ['timonofathens@67357']
riot! ['timonofathens@67550']
Itches, ['timonofathens@67550']
bosoms; ['timonofathens@67597']
bans! ['timonofathens@67812']
woods; ['timonofathens@67856']
confound--hear ['timonofathens@67949']
Athenians ['timonofathens@67998', 'timonofathens@101989']
fall'n!

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



A saída de streaming foi truncada nas últimas 5000 linhas.
profit, ['hamlet@62570', 'cymbeline@141014', 'merchantofvenice@41011', 'periclesprinceoftyre@69892']
inhibition ['hamlet@62620']
estimation ['hamlet@62702', 'coriolanus@54845', 'coriolanus@66261', 'troilusandcressida@47937', 'kinghenryv@67692', '1kinghenryiv@119106', 'allswellthatendswell@119382', 'measureforemeasure@83628', 'muchadoaboutnothing@37397', 'merchantofvenice@101466', 'twogentlemenofverona@31875', 'comedyoferrors@35645']
rusty? ['hamlet@62838']
endeavour ['hamlet@62880', 'troilusandcressida@124909', 'troilusandcressida@158139', '2kinghenryiv@113200', 'kinghenryv@15962', 'kinghenryv@146678', '2kinghenryvi@66697', 'muchadoaboutnothing@37663', 'merchantofvenice@80144', 'twelfthnight@95230', 'periclesprinceoftyre@44903']
pace: ['hamlet@62880', 'muchadoaboutnothing@22681']
children, ['hamlet@62943', 'kingrichardiii@60779', 'kingrichardiii@111062', 'kingrichardiii@140483', 'kingrichardiii@156969', 'coriolanus@127954', 'tr

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



A saída de streaming foi truncada nas últimas 5000 linhas.
stay'd! ['macbeth@8311']
insane ['macbeth@8412']
Cawdor: ['macbeth@9365', 'macbeth@10773']
thane! ['macbeth@9415']
robes? ['macbeth@9592']
capital, ['macbeth@9903']
oftentimes, ['macbeth@10343']
truths, ['macbeth@10383']
theme.--I ['macbeth@10602']
soliciting ['macbeth@10652']
imaginings: ['macbeth@10988']
partner's ['macbeth@11182']
MACBETH, ['macbeth@12738', 'macbeth@14897', 'macbeth@24967', 'macbeth@41974', 'macbeth@42021', 'macbeth@43583', 'macbeth@52665', 'macbeth@52665', 'macbeth@87920', 'macbeth@92291', 'macbeth@96649']
deserved, ['macbeth@12921', 'macbeth@13501']
duties; ['macbeth@13204']
enfold ['macbeth@13548']
Malcolm, ['macbeth@13866', 'macbeth@90734']
Cumberland; ['macbeth@13911']
unaccompanied ['macbeth@13956']
deservers. ['macbeth@14041']
Cumberland! ['macbeth@14313']
Banquo; ['macbeth@14612', 'macbeth@49891']
Macbeth's ['macbeth@14858', 'macbeth@18568', 'macbeth@20288', 'macbeth@24447', 'macbeth@39292']
saluted 